# Demo: Iterative approach to Model-based item-wise collaborative filtering

In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(1, '../resype')
%load_ext autoreload
%autoreload 2 

## Prepare data

In [2]:
movies_df = pd.DataFrame({'M1': [2, None, 3, None, 1, 5],
                          'M2': [None, 1, None, 5, 1, None],
                          'M3': [5, 4, 1, 1, 4, None],
                          'M4': [5, 4, None, None, None, 1]},
                         index=[f'U{i+1}' for i in range(6)])
movies_df

,M1,M2,M3,M4
U1,2.0,NaN,5.0,5.0
U2,NaN,1.0,4.0,4.0
U3,3.0,NaN,1.0,NaN
U4,NaN,5.0,1.0,NaN
U5,1.0,1.0,4.0,NaN
U6,5.0,NaN,NaN,1.0


In [3]:
movie_transactions = pd.melt(
    movies_df.reset_index(), id_vars=['index'], 
    value_vars=movies_df.columns).dropna().reset_index(drop=True)
movie_transactions.columns = ["user_id", 'item_id', 'rating']

In [4]:
movie_transactions

,user_id,item_id,rating
0,U1,M1,2.0
1,U3,M1,3.0
2,U5,M1,1.0
3,U6,M1,5.0
4,U2,M2,1.0
5,U4,M2,5.0
6,U5,M2,1.0
7,U1,M3,5.0
8,U2,M3,4.0
9,U3,M3,1.0


## Load resype

In [5]:
from resype import Resype

In [6]:
re = Resype(movie_transactions)

In [7]:
utility_matrix = re.construct_utility_matrix()
utility_matrix

item_id,M1,M2,M3,M4
user_id,,,,
U1,2.0,NaN,5.0,5.0
U2,NaN,1.0,4.0,4.0
U3,3.0,NaN,1.0,NaN
U4,NaN,5.0,1.0,NaN
U5,1.0,1.0,4.0,NaN
U6,5.0,NaN,NaN,1.0


## Train iterative model using `train_model_iterative`

#### Create model object (load from sklearn)

In [8]:
from sklearn.neural_network import MLPRegressor
rs_model1 = MLPRegressor(hidden_layer_sizes=(100, 50), activation='identity')

#### Train model

In [9]:
re.utility_matrix

item_id,M1,M2,M3,M4
user_id,,,,
U1,2.0,NaN,5.0,5.0
U2,NaN,1.0,4.0,4.0
U3,3.0,NaN,1.0,NaN
U4,NaN,5.0,1.0,NaN
U5,1.0,1.0,4.0,NaN
U6,5.0,NaN,NaN,1.0


In [10]:
utility_matrix_imputed, metrics, trained_model = re.train_model_iterative(
    re.utility_matrix, rs_model1, return_models=True)

#### Prediction

In [11]:
utility_matrix_imputed

item_id,M1,M2,M3,M4
user_id,,,,
U1,-2.000000,-2.039444,1.000000,1.000000
U2,-1.996796,-2.000000,1.000000,1.000000
U3,1.000000,1.355488,-1.000000,-0.974410
U4,1.481669,2.000000,-2.000000,-1.549891
U5,-1.000000,-1.000000,2.000000,0.961755
U6,2.000000,2.547549,-2.786593,-2.000000


#### Convert to scale of original ratings

In [12]:
utility_matrix_imputed.add(re.utility_matrix.mean(axis=1), axis=0)

item_id,M1,M2,M3,M4
user_id,,,,
U1,2.000000,1.960556,5.000000,5.000000
U2,1.003204,1.000000,4.000000,4.000000
U3,3.000000,3.355488,1.000000,1.025590
U4,4.481669,5.000000,1.000000,1.450109
U5,1.000000,1.000000,4.000000,2.961755
U6,5.000000,5.547549,0.213407,1.000000


## Train iterative model using `fit`

#### Create model object (load from sklearn)

In [13]:
rs_model2 = MLPRegressor(hidden_layer_sizes=(50, 50), activation='identity')

#### Train model

In [14]:
re.fit(rs_model2, method='iterative')

#### Prediction

In [15]:
re.utility_matrix_preds

item_id,M1,M2,M3,M4
user_id,,,,
U1,-2.000000,-2.037083,1.00000,1.000000
U2,-1.980248,-2.000000,1.00000,1.000000
U3,1.000000,1.917455,-1.00000,-1.454908
U4,0.933371,2.000000,-2.00000,-1.637402
U5,-1.000000,-1.000000,2.00000,0.849392
U6,2.000000,3.276228,-1.94579,-2.000000
